In [1]:
# Cloning the required files from git directly to avoid pain
!git clone https://github.com/ahmedtarek-/Deep-Q-Learning.git
!cp -r Deep-Q-Learning/* .
!ls

Cloning into 'Deep-Q-Learning'...
remote: Enumerating objects: 116, done.
remote: Counting objects: 100% (116/116), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 116 (delta 52), reused 82 (delta 28), pack-reused 0
Receiving objects: 100% (116/116), 331.55 KiB | 6.38 MiB/s, done.
Resolving deltas: 100% (52/52), done.
Deep-Q-Learning		     gym-grid	requirements.txt
deep_q_learning_arina.ipynb  gym_grid	sample_data
deep_q_learning.ipynb	     README.md	training


In [ ]:
# Install required packages
!pip install -q gym==0.15.4
!pip install -q pycolab==1.2
!pip install -q torch==1.2.0
# !pip install -q matplotlib==3.1.2

In [17]:
# Adding /training to sys path
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('training/')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## A) Simulating the `main.py` file

### 1. Imports and load config

In [18]:
%load_ext autoreload
%autoreload 2

import yaml
from trainer import Trainer
from result import Result

import gym
import gym_grid

with open('training/config.yaml') as f:
  config = yaml.safe_load(f)['training_config']

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 2. Trainer and the loop

In [ ]:
trainer = Trainer(env_name=config["env_name"])

# 4. Run a training loop
training_rewards, training_discounted_rewards = trainer.the_loop()

Streaming output truncated to the last 5000 lines.
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Discounted reward at idx 0 update is 693.0
Intermediate reward at idx 0 update is 700
Dis

### 3. Calling Result

In [ ]:
Result(
  model=trainer.model,
  rewards=training_rewards,
  disc_rewards=training_discounted_rewards,
  env_name=config["env_name"]).save(plot=True)

### 4. Running our simulation

#### 4.1 Method to load env

In [ ]:
import numpy as np
from mlp import MLP

def load_model_env(env, path_to_model):
    states = np.prod(env.observation_space.shape)
    actions = env.action_space.n

    m_load = MLP(states, actions)
    m_load.load_state_dict(torch.load(path_to_model))

    return m_load

#### 4.2 Method to visualise

In [ ]:
def run_trained_model_visual(env, model, num_steps = 20):
    current_state, obs_to_render = env.reset_with_render()
    env.render(obs_to_render)
    rew_total = 0

    # put the model in inference mode
    model.eval()
    done = False

    for _ in range(num_steps):
        action = torch.argmax(model(torch.Tensor(current_state.flatten()))).item()
        current_state, rew , done, _, obs_to_render = env.step_with_render(action)
        env.render(obs_to_render)
        display.display(plt.gcf())
        display.clear_output(wait=True)
        rew_total += rew

        if done:
            break

    print(f"The total reward in this episode is: {rew_total}")